## Load Libraries

In [1]:
import torch
import torch.nn
from torch.optim import optimizer
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Define Transformations

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

## Load the Data

In [5]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_data = datasets.CIFAR10( root="./data", train=True, download=True, transform = transform)
test_data = datasets.CIFAR10( root="./data", train=False, download=True, transform = transform)

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)